# Лабораторная работа №2

**Требования:**
* Python >= 3.X

Лабораторную работу необходимо выполнять в данном шаблоне. Результатом работы будет являться файл (с измененным именем), который необходимо выложить в Moodle.

**Важно!!!** Имя файлу задавайте по следующему шаблону **lab_2_Группа_ФамилияИО.ipynb**. Например: если Вас зовут Иванов Иван Иванович, и Вы обучаетесь в группе 6207_010302D, то имя файла будет выглядеть так **lab_2_6407_010302D_ИвановИИ.ipynb**.

Необходимо провести исследование различных способов представления документов и их влияние на качество определения тональности.

В качестве входных данных к лабораторной работе взят широко известный набор данных IMDB, содержащий 50K обзоров фильмов ([imdb-dataset-of-50k-movie-reviews](https://disk.yandex.ru/i/DDb0zuyUmts5QA)). Откликами являются значения двух классов positive и negative.

In [80]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import cross_val_score

import pandas as pd
import inflect
from nltk.tokenize import RegexpTokenizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup

In [82]:
# Код загрузки данных
# Если хотите добавить какие-либо библиотеки
# добавляйте их ИМЕННО ЗДЕСЬ
nltk.download('punkt')  # Для работы с nltk, если еще не скачано
imdb_data = pd.read_csv(r'input/IMDB Dataset.csv')
imdb_data.head(5)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. Forget pretty pictures painted for mainstream audiences, forget charm, forget romance...OZ doesn't mess around. The first episode I ever saw struck me as so nasty it was surreal, I couldn't say I was ready for it, but as I watched more, I developed a taste for Oz, and got accustomed to the high levels of graphic violence. Not just violence, but injustice (crooked guards who'll be sold out for a nickel, inmates who'll kill on order and get away with it, well mannered, middle class inmates being turned into prison bitches due to their lack of street skills or prison experience) Watching Oz, you may become comfortable with what is uncomfortable viewing....thats if you can get in touch with your darker side.",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only ""has got all the polari"" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master's of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional 'dream' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell's murals decorating every surface) are terribly well done.",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've never been impressed with Scarlet Johanson, in this she managed to tone down her ""sexy"" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than ""Devil Wears Prada"" and more interesting than ""Superman"" a great comedy to go see with friends.",positive
3,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zom

#### Шаг №1 Подготовка данных

Обязательно предобработайте данные!



In [85]:
# Инициализация inflect для чисел
p = inflect.engine()

# Функция для замены чисел их текстовыми представлениями
def replace_numbers_with_words(text):
    tokens = text.split()
    new_tokens = []
    for token in tokens:
        if token.isdigit():
            word_number = p.number_to_words(token)
            new_tokens.append(word_number)
        else:
            new_tokens.append(token)
    return ' '.join(new_tokens)

# Инициализация токенизатора (только слова)
tokenizer = RegexpTokenizer(r'\b\w+\b')

In [87]:
# Отключаем обрезку текста
pd.set_option('display.max_colwidth', None)
imdb_data.iloc[1]

review       A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master's of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional 'dream' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell's murals decorating every surface) are terribly

Присутствуют html шаблоны, поэтому для предобработки была сделана функцию  def clean_html(text). Принимает строку где есть html код передается в html.parser, он обрабатывает html и возвращает нормальный текст


In [90]:
# Функция для очистки HTML-тегов из текста
def clean_html(text):
    return BeautifulSoup(text, "html.parser").get_text()

# Основная функция для предобработки текста
def preprocess_text(text):
    text = clean_html(text)  # Удаляем HTML-теги
    text = text.lower()  # Приводим текст к нижнему регистру
    text = replace_numbers_with_words(text)  # Заменяем числа словами
    tokens = tokenizer.tokenize(text)  # Токенизация
    return tokens

In [92]:
html = "<h1>Вот такой</h1><p> Текст <b>Выводится</b> с HTML-тегами.</p>"

cleaned_text = clean_html(html)  # Вызов функции

print(cleaned_text)


Вот такой Текст Выводится с HTML-тегами.


In [94]:
imdb_data.head(10)

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. Forget pretty pictures painted for mainstream audiences, forget charm, forget romance...OZ doesn't mess around. The first episode I ever saw struck me as so nasty it was surreal, I couldn't say I was ready for it, but as I watched more, I developed a taste for Oz, and got accustomed to the high levels of graphic violence. Not just violence, but injustice (crooked guards who'll be sold out for a nickel, inmates who'll kill on order and get away with it, well mannered, middle class inmates being turned into prison bitches due to their lack of street skills or prison experience) Watching Oz, you may become comfortable with what is uncomfortable viewing....thats if you can get in touch with your darker side.",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only ""has got all the polari"" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master's of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional 'dream' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell's murals decorating every surface) are terribly well done.",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've never been impressed with Scarlet Johanson, in this she managed to tone down her ""sexy"" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than ""Devil Wears Prada"" and more interesting than ""Superman"" a great comedy to go see with friends.",positive
3,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zom

In [96]:
# Применяем предобработку к столбцу 'review'
imdb_data['processed_review'] = imdb_data['review'].apply(preprocess_text)

# Преобразуем список токенов обратно в строку
imdb_data['processed_review_text'] = imdb_data['processed_review'].apply(lambda x: ' '.join(x))


# Сохранение предобработанных данных
output_path = 'output/processed_IMDB_dataset.csv'
imdb_data.to_csv(output_path, index=False)

# Пример: преобразуем текст обратно в строку
imdb_data_sample = imdb_data.sample(frac=0.3, random_state=42)

# Разделение данных на тренировочную и тестовую выборки
X = imdb_data_sample['processed_review_text']
y = imdb_data_sample['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


C:\Users\user\AppData\Local\Temp\ipykernel_2072\2222028589.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(text, "html.parser").get_text()


OSError: Cannot save file into a non-existent directory: 'output'

При векторизации ограничьте количество признаков до 500. В качестве исследуемых способов векторизации текстов необходимо рассмотреть:

#### 1. Компоненты вектора: частоты ([CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)).

In [ ]:
# Векторизация с CountVectorizer
count_vectorizer = CountVectorizer(max_features=500)
X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

#### 2. Компоненты вектора: оценки tf-idf для слова ([TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)).

In [ ]:
# Векторизация с TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=500)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

#### 3. Компоненты вектора: частоты N-грам.

In [ ]:
# Векторизация с N-граммами
ngram_vectorizer = CountVectorizer(ngram_range=(1, 3), max_features=500)
X_train_ngram = ngram_vectorizer.fit_transform(X_train)
X_test_ngram = ngram_vectorizer.transform(X_test)

### Шаг 2. Исследование моделей

Матрица ошибок (confusion matrix):

<table>
		<tr>
			<td></td>
			<td>$y = 1$</td>
			<td>$y = 0$</td>
		</tr>
		<tr>
			<td>$a(x) = 1$</td>
			<td>True Positive (TP)</td>
			<td>False Positive (FP)</td>
		</tr>
    	<tr>
			<td>$a(x) = 0$</td>
			<td>False Negative (FN)</td>
			<td>True Negative (TN)</td>
		</tr>
</table>

Метрики качества классификации:

$$\operatorname{accuracy} = \frac{\operatorname{TP} + \operatorname{TN}}{\operatorname{TP} + \operatorname{TN} + \operatorname{FP} + \operatorname{FN}}$$


$$\operatorname{precision} = \frac{\operatorname{TP}}{\operatorname{TP} + \operatorname{FP}}$$

$$\operatorname{recall} = \frac{\operatorname{TP}}{\operatorname{TP} + \operatorname{FN}}$$

$$\operatorname{F} = \frac{\operatorname{precision} \cdot \operatorname{recall}}{\operatorname{precision} + \operatorname{recall}}$$

Для каждой модели и каждого способа векторизации текстов необходимо:

1. Определить оптимальные гиперпараметры (по F-мере) ([GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)).

2. Количество блоков при перекрестной проверке (cross-validation) должно быть равно 3.

3. Для ускорения процесса можно ограничиться 30% всех данных.

Оценку производим для следующих моделей:

#### 1. Машина опорных векторов ([SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)).

In [68]:
# Сетку гиперпараметров для SVC
param_grid_svc = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

**TF-IDF**

In [71]:
# Обучение модели SVC для TF-IDF
svc_model = SVC(random_state=42)
grid_search_svc = GridSearchCV(svc_model, param_grid_svc, scoring='f1', cv=3, n_jobs=-1)
grid_search_svc.fit(X_train_tfidf, y_train)
svc_best_tfidf = grid_search_svc.best_estimator_
y_pred_svc_tfidf = svc_best_tfidf.predict(X_test_tfidf)

# Оценка модели SVC для TF-IDF
print("Метрики качества для SVC (TF-IDF):")
print(classification_report(y_test, y_pred_svc_tfidf))
print(confusion_matrix(y_test, y_pred_svc_tfidf))

NameError: name 'X_train_tfidf' is not defined

In [73]:
# Вывод лучших параметров для модели SVC
print("Лучшие параметры для SVC (TF-IDF):")
print(grid_search_svc.best_params_)

Лучшие параметры для SVC (TF-IDF):


AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

**CountVectorizer**

In [76]:
# Сетку гиперпараметров для SVC
param_grid_svc1 = {
    'C': [0.1],
    'kernel': ['linear'],
    'gamma': ['scale']
}

In [78]:
# Обучение модели SVC для CountVectorizer
grid_search_svc_count = GridSearchCV(svc_model, param_grid_svc1, scoring='f1', cv=3, n_jobs=-1)
grid_search_svc_count.fit(X_train_count, y_train)
svc_best_count = grid_search_svc_count.best_estimator_
y_pred_svc_count = svc_best_count.predict(X_test_count)

# Оценка модели SVC для CountVectorizer
print("Метрики качества для SVC (CountVectorizer):")
print(classification_report(y_test, y_pred_svc_count))
print(confusion_matrix(y_test, y_pred_svc_count))

NameError: name 'X_train_count' is not defined

In [25]:
# Вывод лучших параметров для модели SVC (CountVectorizer)
print("Лучшие параметры для SVC (CountVectorizer):")
print(grid_search_svc_count.best_params_)

Лучшие параметры для SVC (CountVectorizer):
{'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}


**N-грамм**

In [19]:
# Обучение модели SVC для N-грамм
grid_search_svc_ngram = GridSearchCV(svc_model, param_grid_svc, scoring='f1', cv=3, n_jobs=-1)
grid_search_svc_ngram.fit(X_train_ngram, y_train)
svc_best_ngram = grid_search_svc_ngram.best_estimator_
y_pred_svc_ngram = svc_best_ngram.predict(X_test_ngram)

# Оценка модели SVC для N-грамм
print("Метрики качества для SVC (N-граммы):")
print(classification_report(y_test, y_pred_svc_ngram))
print(confusion_matrix(y_test, y_pred_svc_ngram))

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:1103: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


Метрики качества для SVC (N-граммы):
              precision    recall  f1-score   support

    negative       0.83      0.80      0.82      2238
    positive       0.81      0.84      0.82      2262

    accuracy                           0.82      4500
   macro avg       0.82      0.82      0.82      4500
weighted avg       0.82      0.82      0.82      4500

[[1792  446]
 [ 361 1901]]


In [20]:
# Вывод лучших параметров для модели SVC (N-граммы)
print("Лучшие параметры для SVC (N-граммы):")
print(grid_search_svc_ngram.best_params_)

Лучшие параметры для SVC (N-граммы):
{'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}


#### 2. Случайный лес ([RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)).

In [26]:
# Сетку гиперпараметров для RandomForest
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 10]
}

**TF-IDF**

In [27]:
# Обучение модели RandomForest для TF-IDF
rf_model = RandomForestClassifier(random_state=42)
grid_search_rf = GridSearchCV(rf_model, param_grid_rf, scoring='f1', cv=3, n_jobs=-1)
grid_search_rf.fit(X_train_tfidf, y_train)
rf_best_tfidf = grid_search_rf.best_estimator_
y_pred_rf_tfidf = rf_best_tfidf.predict(X_test_tfidf)

# Оценка модели RandomForest для TF-IDF
print("Метрики качества для RandomForest (TF-IDF):")
print(classification_report(y_test, y_pred_rf_tfidf))
print(confusion_matrix(y_test, y_pred_rf_tfidf))

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:1103: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


Метрики качества для RandomForest (TF-IDF):
              precision    recall  f1-score   support

    negative       0.80      0.79      0.79      2238
    positive       0.79      0.81      0.80      2262

    accuracy                           0.80      4500
   macro avg       0.80      0.80      0.80      4500
weighted avg       0.80      0.80      0.80      4500

[[1763  475]
 [ 440 1822]]


In [31]:
# Вывод лучших параметров для модели RandomForest (TF-IDF)
print("Лучшие параметры для RandomForest (TF-IDF):")
print(grid_search_rf.best_params_)

Лучшие параметры для RandomForest (TF-IDF):
{'max_depth': None, 'min_samples_split': 2, 'n_estimators': 50}


**CountVectorizer**

In [28]:
# Обучение модели RandomForest для CountVectorizer
grid_search_rf_count = GridSearchCV(rf_model, param_grid_rf, scoring='f1', cv=3, n_jobs=-1)
grid_search_rf_count.fit(X_train_count, y_train)
rf_best_count = grid_search_rf_count.best_estimator_
y_pred_rf_count = rf_best_count.predict(X_test_count)

# Оценка модели RandomForest для CountVectorizer
print("Метрики качества для RandomForest (CountVectorizer):")
print(classification_report(y_test, y_pred_rf_count))
print(confusion_matrix(y_test, y_pred_rf_count))

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:1103: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


Метрики качества для RandomForest (CountVectorizer):
              precision    recall  f1-score   support

    negative       0.80      0.77      0.79      2238
    positive       0.78      0.81      0.80      2262

    accuracy                           0.79      4500
   macro avg       0.79      0.79      0.79      4500
weighted avg       0.79      0.79      0.79      4500

[[1734  504]
 [ 432 1830]]


In [30]:
print("Лучшие параметры для RandomForest (CountVectorizer):")
print(grid_search_rf_count.best_params_)

Лучшие параметры для RandomForest (CountVectorizer):
{'max_depth': None, 'min_samples_split': 2, 'n_estimators': 50}


**N-грамм**

In [29]:
# Обучение модели RandomForest для N-грамм
grid_search_rf_ngram = GridSearchCV(rf_model, param_grid_rf, scoring='f1', cv=3, n_jobs=-1)
grid_search_rf_ngram.fit(X_train_ngram, y_train)
rf_best_ngram = grid_search_rf_ngram.best_estimator_
y_pred_rf_ngram = rf_best_ngram.predict(X_test_ngram)

# Оценка модели RandomForest для N-грамм
print("Метрики качества для RandomForest (N-граммы):")
print(classification_report(y_test, y_pred_rf_ngram))
print(confusion_matrix(y_test, y_pred_rf_ngram))

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:1103: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


Метрики качества для RandomForest (N-граммы):
              precision    recall  f1-score   support

    negative       0.78      0.76      0.77      2238
    positive       0.77      0.78      0.78      2262

    accuracy                           0.77      4500
   macro avg       0.77      0.77      0.77      4500
weighted avg       0.77      0.77      0.77      4500

[[1705  533]
 [ 488 1774]]


In [32]:
print("Лучшие параметры для RandomForest (N-граммы):")
print(grid_search_rf_ngram.best_params_)

Лучшие параметры для RandomForest (N-граммы):
{'max_depth': None, 'min_samples_split': 2, 'n_estimators': 50}


### Шаг 3. Сравнение результатов

1. Сравнить и найти наиболее точные (по F-мере) модель и способ векторизации текстов.

2. Обучить полученную модель (с заданными оптимальными гиперпараметрами) на всех данных (80% обучающая выборка, 20% тестовая выборка).

3. На тестовой выборке постоить матрицу ошибок (confusion matrix) и оценить качество классификации как accuracy, precision, recall и F-мера.

Лучшие параметры для SVC (TF-IDF):
{'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}

In [33]:
# Загружаем все данные
imdb_data_full = pd.read_csv('output/processed_IMDB_dataset.csv')

# Применяем предобработку
imdb_data_full['processed_review_text'] = imdb_data_full['processed_review'].apply(lambda x: ' '.join(eval(x)))

# Определяем целевую переменную
y_full = imdb_data_full['sentiment']  # метка (0 - отрицательный, 1 - положительный)
X_full = imdb_data_full['processed_review_text']  # текстовые данные

# Разделение данных на тренировочную и тестовую выборки (80% обучающая, 20% тестовая)
X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(X_full, y_full, test_size=0.2, random_state=42)


In [36]:
imdb_data_full

,review,sentiment,processed_review,processed_review_text
0,One of the other reviewers has mentioned that ...,positive,"['one', 'of', 'the', 'other', 'reviewers', 'ha...",one of the other reviewers has mentioned that ...
1,A wonderful little production. <br /><br />The...,positive,"['a', 'wonderful', 'little', 'production', 'th...",a wonderful little production the filming tech...
2,I thought this was a wonderful way to spend ti...,positive,"['i', 'thought', 'this', 'was', 'a', 'wonderfu...",i thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...,negative,"['basically', 'there', 's', 'a', 'family', 'wh...",basically there s a family where a little boy ...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"['petter', 'mattei', 's', 'love', 'in', 'the',...",petter mattei s love in the time of money is a...
...,...,...,...,...
49995,I thought this movie did a down right good job...,positive,"['i', 'thought', 'this', 'movie', 'did', 'a', ...",i thought this movie did a down right good job...
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,"['bad', 'plot', 'bad', 'dialogue', 'bad', 'act...",bad plot bad dialogue bad acting idiotic direc...
49997,I am a Catholic taught in parochial elementary...,negative,"['i', 'am', 'a', 'catholic', 'taught', 'in', '...",i am a catholic taught in parochial elementary...
49998,I'm going to have to disagree with the previou...,negative,"['i', 'm', 'going', 'to', 'have', 'to', 'disag...",i m going to have to disagree with the previou...


In [34]:
# Векторизация текста с использованием TF-IDF (ограничиваем 500 признаками)
tfidf_vectorizer_full = TfidfVectorizer(max_features=500)
X_train_tfidf_full = tfidf_vectorizer_full.fit_transform(X_train_full)
X_test_tfidf_full = tfidf_vectorizer_full.transform(X_test_full)

In [35]:
# Инициализация модели SVC с лучшими параметрами
svc_best_model = SVC(C=0.1, gamma='scale', kernel='linear', random_state=42)

# Обучение модели на всех данных
svc_best_model.fit(X_train_tfidf_full, y_train_full)

# Прогнозирование на тестовой выборке
y_pred_full = svc_best_model.predict(X_test_tfidf_full)

# Оценка модели
print("Метрики качества для SVC (на всех данных):")
print(classification_report(y_test_full, y_pred_full))
print(confusion_matrix(y_test_full, y_pred_full))

Метрики качества для SVC (на всех данных):
              precision    recall  f1-score   support

    negative       0.85      0.82      0.84      4961
    positive       0.83      0.86      0.84      5039

    accuracy                           0.84     10000
   macro avg       0.84      0.84      0.84     10000
weighted avg       0.84      0.84      0.84     10000

[[4073  888]
 [ 707 4332]]
